In [ ]:
!pip install webdataset google-cloud-storage torch torchvision


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [17]:
!export GOOGLE_APPLICATION_CREDENTIALS="/content/drive/MyDrive/checkpoints/ldm100k-dl-95af770e42ef.json"

In [ ]:
!pip install nibabel


In [ ]:
!gcloud auth login --no-launch-browser

In [ ]:
!gcloud config set project ldm100k

In [ ]:
!gcloud storage ls gs://ldm100k-bucket

In [ ]:
!gsutil cat gs://ldm100k-bucket/shards/shard-0000000.tar

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

In [25]:
mkdir -p /content/mnt/ldm100k

In [ ]:
!gcsfuse ldm100k-bucket /content/mnt/ldm100k

In [ ]:
# persistent mount on VM
ldm100k-bucket /mnt/ldm100k gcsfuse rw,noatime,key_file=/path/to/your/key.json 0 0
# need to doublecheck if mount point persists through shutdown/machine swap

In [28]:
import os
len(os.listdir("/content/mnt/ldm100k/shards"))

3127

In [18]:
import os

# This sets the environment variable for the entire Python process (and subprocesses like gsutil, gcsfuse)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/checkpoints/ldm100k-dl-95af770e42ef.json"


In [36]:
dataset = wds.WebDataset("/content/mnt/ldm100k/shards/shard-*.tar")


In [ ]:
import glob
sorted(glob.glob("/content/mnt/ldm100k/shards/shard-*.tar"))


In [88]:
import torch
import webdataset as wds
import glob
import nibabel as nib
import json
import io

def decode_nii_gz(sample):
    fileobj = io.BytesIO(sample)
    img = nib.load(fileobj)
    data = img.get_fdata()
    return data.astype("float32")

def decode_json(sample):
    return json.loads(sample.decode("utf-8"))

shard_paths = sorted(glob.glob("/content/mnt/ldm100k/shards/shard-*.tar"))

dataset = (
    wds.WebDataset(shard_paths, handler=wds.ignore_and_continue, shardshuffle=False)
)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, num_workers=2)

In [89]:
sample = next(iter(dataset))

In [90]:
sample = next(iter(dataloader))

In [91]:
sample.keys()

dict_keys(['__key__', '__url__', 'image.nii.gz', '__local_path__', 'json'])

In [92]:
sample["json"]

[b'{"age": 59.16975259780884, "sex": "F", "ventricular_volume": 46664.72856965304, "brain_volume": 1756032.3683834076}',
 b'{"age": 71.92763137817383, "sex": "M", "ventricular_volume": 155847.29660019875, "brain_volume": 1205682.300041318}',
 b'{"age": 52.54857397079468, "sex": "M", "ventricular_volume": 154541.76321305273, "brain_volume": 1580107.3121392727}',
 b'{"age": 79.73708689212799, "sex": "M", "ventricular_volume": 140314.50815726042, "brain_volume": 1195746.585561633}']

In [156]:
sample["__key__"]

['00000266', '00000042', '00000391', '00000018']

In [121]:
import nibabel as nib
import gzip
import io
import torch

def decode_batch_nii(batch_bytes):
    batch = []
    for b in batch_bytes:
        # Decompress the .nii.gz file from bytes
        decompressed = gzip.decompress(b)

        # Wrap decompressed NIfTI data in BytesIO
        fileobj = io.BytesIO(decompressed)

        # Use nibabel's low-level filemap interface
        file_map = nib.Nifti1Image.make_file_map()
        file_map["image"].fileobj = fileobj
        file_map["image"].pos = 0

        # Actually load the NIfTI object
        img = nib.Nifti1Image.from_file_map(file_map)

        # Convert to torch tensor
        arr = img.get_fdata().astype("float32")
        batch.append(torch.from_numpy(arr))

    return torch.stack(batch)

In [ ]:
sample = next(iter(dataloader))

image_bytes = sample["image.nii.gz"]  # list of byte streams
img_bytes = image_bytes[0]
type(img_bytes)

decode_batch_nii(image_bytes)

In [112]:
with gzip.open(BytesIO(img_bytes), "rb") as gz:
  decompressed = gz.read()

In [118]:
type(decompressed)
print(decompressed[0:20])

b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\x03\xec\xbb\x05P\x1d[\x9b\xa8\xbd!'


In [150]:
with gzip.open(BytesIO(img_bytes), "rb") as gz:
  decompressed = gz.read()
  #img = nib.load(decompressed)

In [151]:
def save_nii_gz(bytes_data: bytes, output_path: str):
    with open(output_path, 'wb') as f:
        f.write(bytes_data)

# Example usage
save_nii_gz(decompressed, 'output_file.nii.gz')


In [152]:
xd = nib.load("output_file.nii.gz")

In [153]:
img_data = xd.get_fdata()

In [160]:
xd

In [ ]:
!pip install tensorhue

In [144]:
from tensorhue import viz

In [154]:
img_data.shape

(160, 224, 160)

In [ ]:
from nibabel import FileHolder, Nifti1Image
bb = BytesIO(decompressed)
fh = FileHolder(fileobj=bb)
img = Nifti1Image.from_file_map({"header": fh, "image": fh})

In [163]:
# for gzipped files
from gzip import GzipFile
fh = FileHolder(fileobj=GzipFile(fileobj=BytesIO(decompressed)))
img = Nifti1Image.from_file_map({"header": fh, "image": fh})

In [165]:
dta = img.get_fdata()

In [166]:
dta.shape

(160, 224, 160)

In [ ]:
# https://stackoverflow.com/questions/68627414/how-to-convert-binary-file-to-nifti

In [168]:
img_slice = dta[:, 110, :]
img_slice.shape

(160, 160)

In [170]:
from scipy.ndimage import zoom

# For 2D array, zoom factor 0.5 reduces size by half
subsampled = zoom(img_slice, zoom=0.5, order=1)  # order=1 is bilinear interpolation


In [172]:
viz(subsampled)

▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀
▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀
▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀
▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀
▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀
▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀
▀▀▀▀▀▀▀▀▀▀▀▀▀▀[38;2;80;18;